In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import signal
%matplotlib inline

#### Import the open/close time data.

In [15]:
df = pd.read_csv(filepath_or_buffer = 'data/cham1_coll_close_times.csv', index_col = False)
print df.head()

   year  month  day  hour  min  sec  close_hour  cmin  csec  open_hour  omin  \
0  2014      7   18    16   16   27          16    38     0         16    37   
1  2014      7   18    17   18   41          17    40    19         17    39   
2  2014      7   18    18   21    1          18    42    39         18    42   
3  2014      7   18    19   23   21          19    45     0         19    44   
4  2014      7   18    20   25   42          20    47    20         20    46   

   osec  camb  close_month  close_day  
0    29     1            7         18  
1    49     1            7         18  
2     9     1            7         18  
3    29     1            7         18  
4    50     1            7         18  


In [16]:
df['sec'] = df['sec'].map(lambda x: 0 if x == 60 else x)
def f(x): return round(x)
df['sec'] = map(f, df['sec'])
df['closing'] = pd.to_datetime(df.year.astype(str) + '-' + df.month.astype(str) + '-' + df.day.astype(str) + ' ' + df.open_hour.astype(str) + ':' + df['omin'].astype(str) + ':' + df.osec.astype(str))
df['closed'] = pd.to_datetime(df.year.astype(str) + '-' + df.month.astype(str) + '-' + df.day.astype(str) + ' ' + df.close_hour.astype(str) + ':' + df['cmin'].astype(str) + ':' + df.csec.astype(str))
df['end_of_run'] = pd.to_datetime(df.year.astype(str) + '-' + df.month.astype(str) + '-' + df.day.astype(str) + ' ' + df.hour.astype(str) + ':' + df['min'].astype(str) + ':' + df.sec.astype(str))
df = df.set_index(pd.DatetimeIndex(df['end_of_run']))

In [17]:
df.head()

,year,month,day,hour,min,sec,close_hour,cmin,csec,open_hour,omin,osec,camb,close_month,close_day,closing,closed,end_of_run
2014-07-18 16:16:27,2014,7,18,16,16,27,16,38,0,16,37,29,1,7,18,2014-07-18 16:37:29,2014-07-18 16:38:00,2014-07-18 16:16:27
2014-07-18 17:18:41,2014,7,18,17,18,41,17,40,19,17,39,49,1,7,18,2014-07-18 17:39:49,2014-07-18 17:40:19,2014-07-18 17:18:41
2014-07-18 18:21:01,2014,7,18,18,21,1,18,42,39,18,42,9,1,7,18,2014-07-18 18:42:09,2014-07-18 18:42:39,2014-07-18 18:21:01
2014-07-18 19:23:21,2014,7,18,19,23,21,19,45,0,19,44,29,1,7,18,2014-07-18 19:44:29,2014-07-18 19:45:00,2014-07-18 19:23:21
2014-07-18 20:25:42,2014,7,18,20,25,42,20,47,20,20,46,50,1,7,18,2014-07-18 20:46:50,2014-07-18 20:47:20,2014-07-18 20:25:42


#### Import the reference data.

In [18]:
df_R = pd.read_csv(filepath_or_buffer = 'data/coll_ref.csv', parse_dates = [0], index_col = 0)
df_R.head()

IOError: File data/coll_ref.csv does not exist

In [6]:
df['start'] = df.closing - pd.Timedelta(minutes=10)
df['stop'] = df.closed + pd.Timedelta(minutes=10)

#### Use this code to get closed data points

In [7]:
# df_new = pd.DataFrame({'o3_ppb':[]})
# date_list = []
# o3_list = []
# for row_index, row in df.iterrows():
#     for row_index_ref, row_ref in df_R.iterrows():
#         if row_index_ref >= (row.closed + pd.Timedelta(minutes=4)) and row_index_ref <= (row.closed + pd.Timedelta(minutes=7)):
#             date_list.append(row_index_ref)
#             o3_list.append(int(row_ref.e2v03))

#### Use this for open data points

In [8]:
df_new = pd.DataFrame({'o3_ppb':[]})
date_list = []
o3_list = []
for row_index, row in df.iterrows():
    for row_index_ref, row_ref in df_R.iterrows():
        if row_index_ref >= (row.closing - pd.Timedelta(minutes=10)) and row_index_ref <= (row.closing - pd.Timedelta(minutes=3)):
            date_list.append(row_index_ref)
            o3_list.append(int(row_ref.e2v03))

In [9]:
df_new.o3_ppb = o3_list
df_new['date'] = date_list

In [10]:
df_new.head()

,o3_ppb,date


In [109]:
df_R['date'] = df_R.index

In [110]:
df_final = pd.merge(df_new, df_R, how='inner', on='date')

In [111]:
df_final.head()

,o3_ppb,date,e2v03,no2,Temp,Rh,ln_O3,ln_temp,ln_rh,e2v03_int_lag_1,...,Temp_sq,Temp_interact_ln_temp,Temp_interact_Rh,Temp_interact_ln_rh,ln_temp_sq,ln_temp_interact_Rh,ln_temp_interact_ln_rh,Rh_sq,Rh_interact_ln_rh,ln_rh_sq
0,677,2014-07-21 14:45:59,677,848,51.1,12.0,6.517671,3.933784,2.484907,0,...,2611.21,201.016388,613.20,126.978730,15.474660,47.205414,9.775087,144.00,29.818880,6.174761
1,677,2014-07-21 14:46:03,677,848,51.1,12.0,6.517671,3.933784,2.484907,0,...,2611.21,201.016388,613.20,126.978730,15.474660,47.205414,9.775087,144.00,29.818880,6.174761
2,677,2014-07-21 14:46:08,677,848,51.2,12.0,6.517671,3.935740,2.484907,0,...,2621.44,201.509864,614.40,127.227220,15.490046,47.228874,9.779945,144.00,29.818880,6.174761
3,677,2014-07-21 14:46:13,677,848,51.1,12.0,6.517671,3.933784,2.484907,0,...,2611.21,201.016388,613.20,126.978730,15.474660,47.205414,9.775087,144.00,29.818880,6.174761
4,678,2014-07-21 14:46:18,678,848,51.2,12.1,6.519147,3.935740,2.493205,0,...,2621.44,201.509864,619.52,127.652119,15.490046,47.622448,9.812607,146.41,30.167786,6.216073


In [112]:
df_final.drop('o3_ppb', axis=1, inplace=True)


In [113]:
df_final.set_index('date', inplace=True)

In [114]:
df_final.to_csv(path_or_buf = 'data/cham_3_open_only.csv')